# Automated Bug Priority Predictor

## Overview
This project aims to develop a machine learning model that predicts the **priority level of software bug reports** based on their text descriptions and metadata.
By automating the triage process, software teams can save time and focus on resolving critical issues faster.

## Stack
- **Frontend:** Streamlit
- **Backend:** FastAPI
- **Database:** SQLite wit SQLAlchemy ORM
- **ML Environment:** Python (Jupyter Notebook, scikit-learn, pandas, nltk)
- **Datasets:** GitHub Issues Dataset and GitBugs/ms_vscode_bugs (Kaggle)
  - GitHub Issues: https://www.kaggle.com/datasets/davidshinn/github-issues?resource=download
  - GitBugs: https://www.kaggle.com/datasets/av9ash/gitbugs/data


---

# 01 Exploration Data Analysis

## Objective
Explore and clean the data to prepare it for model training in the next phase.

## Notebook Goals
1. Load and inspect the dataset.
2. Understand column types and distribution.
3. Handle missing values and duplicates.
4. Clean and normalize text fields (tokenization, stopwords, lemmatization).
5. Prepare a clean dataset for feature extraction and modeling

## Imports and Setup

In [1]:
# Standard data libraries
import pandas as pd
import numpy as np

# Visualization
from wordcloud import WordCloud
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

# Text processing
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Export precessed DataFrame
import joblib

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

# Download required NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rusty\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rusty\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Load Data

In [2]:
def load_data_sample(filename, n_sample=25000, chunksize=50000, random_state=42):
  """
  Samples n_sample rows from a large CSV without loading the full file into memory.

  Args:
      filename (str): path to CSV file
      n_sample (int, optional): Number of rows to sample. Defaults to 25000.
      chunksize (int, optional): Number of rows to read at a time. Defaults to 50000.
      random_state (int, optional): For reproducibility. Defaults to 42.
  """
  
  sampled_rows = []
  
  for chunk in pd.read_csv(filename, chunksize=chunksize):
    # Fraction to sample from current chunk
    frac = n_sample / len(chunk)
    sampled_rows.append(chunk.sample(frac=min(1, frac), random_state=random_state))
    
  sampled_df = pd.concat(sampled_rows).sample(n=n_sample, random_state=random_state).reset_index(drop=True)
  
  return sampled_df

In [3]:
# Github Issues
git_issues_filename = "../../data/raw/github_issues.csv"

git_issues_df = load_data_sample(git_issues_filename, n_sample=50000)
print("Github Issues shape: ", git_issues_df.shape)

Github Issues shape:  (50000, 3)


In [4]:
# GitBugs
gitbugs_filename = "../../data/raw/gitbugs/ms_vscode_bugs/vscode_bugs.csv"

gitbugs_df = load_data_sample(gitbugs_filename, n_sample=25000)
print("GitBugs shape: ", gitbugs_df.shape)

GitBugs shape:  (25000, 6)


### Data Exploration

In [5]:
git_issues_df.head()

,issue_url,issue_title,body
0,"""https://github.com/LeoTindall/randomua/issues/2""",i can't reset the user agent,"after i installed randomua i can't download some add-ons from amo since amo now thinks i use firefox 52. i tried excluding addons.mozilla.org didn't work , setting the minimum spoofer version to 5..."
1,"""https://github.com/meyersbs/SPLAT/issues/23""",long strings of input on the command-line fail,long strings of input on the command-line cause a failure because those strings are too long to be a filename. - truncate the text of long inputs when creating the filename. - add an option to spe...
2,"""https://github.com/LaserWeb/LaserWeb4/issues/170""",raster simulation rendering,! lw4_raster_display-simulate_quirks https://cloud.githubusercontent.com/assets/2752900/23263874/bf751316-f99c-11e6-827e-fee53d30c0d6.gif lw4 is a huge improvement over lw3 and i am appreciative o...
3,"""https://github.com/tryashtar/minecraft-bugs/issues/313""",tab key not working for any function on mac,mojira ticket mc-334 https://bugs.mojang.com/browse/mc-334 <img src= https://secure.gravatar.com/avatar/c73b06d0f7d78154de104f2e546091db?d=mm&s=24 width=20 height=20> newell ellis https://bugs.moj...
4,"""https://github.com/MarlinFirmware/Marlin/issues/7968""",first reading from adc without adc start adc in temperature.cpp,"temperature.cpp , version 1.1.x and 2.0 in fuction void temperature::isr static adcsensorstate adc_sensor_state = startupdelay; cpp switch adc_sensor_state { ...... if has_temp_0 case preparetemp_..."


In [6]:
git_issues_df.tail()

,issue_url,issue_title,body
49995,"""https://github.com/celynwalters/cmus/issues/3""",show encodedby tag in library view,not sure if this is easy to do...
49996,"""https://github.com/rthotakura97/decentralized-database/issues/6""",asynch handling of requests,"currently, any requests received by our server handles them synchronously. we should make the handler as fast as possible and create a new thread to handle the actual logic. this new thread will n..."
49997,"""https://github.com/recharts/recharts/issues/927""",specify arbitrary domain for the cartesian x and y axes,"what is the current behavior? currently the domain accepts numbers, 'auto', or addition/subtraction from datamin and datamax. what is the expected behavior? i would like to be able to pass functio..."
49998,"""https://github.com/animenotifier/notify.moe/issues/46""",editform: bool type,add support for bool types in utils.editform .
49999,"""https://github.com/angular/angular-cli/issues/3968""",bug build --prod doesn't produce inline.hash.bundle.js.gz,"hi, ng build --prod --aot doesn't produce a gzip version of the inline.hash.bundle.js file. other files are gzip'd as expected. versions. angular-cli: 1.0.0-beta.24 node: 7.3.0 os: linux x64 @angu..."


In [7]:
git_issues_df.info()
git_issues_df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   issue_url    50000 non-null  object
 1   issue_title  50000 non-null  object
 2   body         50000 non-null  object
dtypes: object(3)
memory usage: 1.1+ MB


,issue_url,issue_title,body
count,50000,50000,50000
unique,49990,48415,48661
top,"""https://github.com/cmty-test/cmty-repository-test/issues/2""",first from flow in uk south,first from flow in uk south
freq,4,833,833


## Handling missing values and duplicates

In [8]:
git_issues_df.isna().sum()

issue_url      0
issue_title    0
body           0
dtype: int64

In [9]:
duplicates = git_issues_df[git_issues_df.duplicated(subset='body', keep=False)]
print("Number of duplicate rows: ", duplicates.shape[0])

Number of duplicate rows:  1536


In [10]:
duplicates[['issue_title', 'body']].head(10)

,issue_title,body
11,missing card: -coin token set by ball,check if this card's info exists in deckdata.js
20,please add me - champs-sur-marne,hello! :wave: please add me as a collaborator to this repo.
36,first from flow in uk south,first from flow in uk south
40,first from flow in uk south,first from flow in uk south
54,first from flow in uk south,first from flow in uk south
57,first from flow in uk south,first from flow in uk south
63,contribute to the kill list,"to add a bit of a personal touch, add a programmer grievance of your own to the kill_list.md file."
64,first from flow in uk south,first from flow in uk south
121,missing card: -coin token set by bridge troll,check if this card's info exists in deckdata.js
172,5.1 program structure ~ 3 hrs,5.1 program structure ~ 3 hrs\neloquent javascript is a fantastic free book that would be great to eventually read in depth. let's jump into the second chapter. - start your toggl timer.\n- read c...


In [11]:
git_issues_df = git_issues_df.drop_duplicates(subset='body', keep='first')
print("Dataset shape after removing duplicates: ", git_issues_df.shape)

Dataset shape after removing duplicates:  (48661, 3)


## Challenge: Missing Priority Labels

one key challenge with the GitHub Issues dataset is that it **does not contain explicit priority or severity labels** (e.g., Critical, High, Medium, Low).

- **Impact:** Without a target variable, a supervised machine learning model cannot directly learn to classify issues by priority.
- **Common in real-word data:** Many open-source and internal issue trackers do not consistently label bugs with priority. This is a realistic scenario or software teams.

#### Solution: Weak Supervision via NLP

To address this, I will generate **pseudo-labels** for priority using **keyword-based NLP heuristics** on the issue text (`issue_title + body`):

- Define priority-related keywords for each class (Critical, High, Medium, Low).
- Assign the highest matching priority found in the text to each issue.
- Treat these derived labels as training targets for the ML model.

> Note: These labels are noisy approximations, but they allow me to **demonstrate the end-to-end ML pipeline** (text preprocessing -> feature extraction -> classification -> evaluation) in a realistic setting. Future work could involve human-labeled data or more sophisticated weak supervision techniques.

In [12]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [13]:
def clean_text(text):
  """
  Cleans and preprocess text

  Args:
      text (str): The text that gets cleaned
  """
  
  text = str(text).lower()
  text = text.replace('_', ' ')
  text = re.sub(r"http\S+|www\S+|https\S+", "", text)   # removes urls
  text = re.sub(r"[^a-z0-9\s]", "", text)               # removes punctuation
  tokens = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]
  return " ".join(tokens)

In [14]:
combined_text = git_issues_df['issue_title'].fillna('') + ' ' + git_issues_df['body'].fillna('')
git_issues_df['clean_text'] = combined_text.apply(clean_text)

git_issues_df.head()

,issue_url,issue_title,body,clean_text
0,"""https://github.com/LeoTindall/randomua/issues/2""",i can't reset the user agent,"after i installed randomua i can't download some add-ons from amo since amo now thinks i use firefox 52. i tried excluding addons.mozilla.org didn't work , setting the minimum spoofer version to 5...",cant reset user agent installed randomua cant download addons amo since amo think use firefox 52 tried excluding addonsmozillaorg didnt work setting minimum spoofer version 53 didnt work deactivat...
1,"""https://github.com/meyersbs/SPLAT/issues/23""",long strings of input on the command-line fail,long strings of input on the command-line cause a failure because those strings are too long to be a filename. - truncate the text of long inputs when creating the filename. - add an option to spe...,long string input commandline fail long string input commandline cause failure string long filename truncate text long input creating filename add option specify filename
2,"""https://github.com/LaserWeb/LaserWeb4/issues/170""",raster simulation rendering,! lw4_raster_display-simulate_quirks https://cloud.githubusercontent.com/assets/2752900/23263874/bf751316-f99c-11e6-827e-fee53d30c0d6.gif lw4 is a huge improvement over lw3 and i am appreciative o...,raster simulation rendering lw4 raster displaysimulate quirk lw4 huge improvement lw3 appreciative volunteer work gone contribute code paid nothing use program right criticism therefore intended c...
3,"""https://github.com/tryashtar/minecraft-bugs/issues/313""",tab key not working for any function on mac,mojira ticket mc-334 https://bugs.mojang.com/browse/mc-334 <img src= https://secure.gravatar.com/avatar/c73b06d0f7d78154de104f2e546091db?d=mm&s=24 width=20 height=20> newell ellis https://bugs.moj...,tab key working function mac mojira ticket mc334 img src width20 height20 newell elli oct 24 2012 mac tab key anything ex player list autofinish command player list key bound another key player li...
4,"""https://github.com/MarlinFirmware/Marlin/issues/7968""",first reading from adc without adc start adc in temperature.cpp,"temperature.cpp , version 1.1.x and 2.0 in fuction void temperature::isr static adcsensorstate adc_sensor_state = startupdelay; cpp switch adc_sensor_state { ...... if has_temp_0 case preparetemp_...",first reading adc without adc start adc temperaturecpp temperaturecpp version 11x 20 fuction void temperatureisr static adcsensorstate adc sensor state startupdelay cpp switch adc sensor state tem...


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

texts = git_issues_df['clean_text']

v = TfidfVectorizer(
  max_features=10000,
  ngram_range=(1, 2),
  min_df=5,
  max_df=0.9
)

transformed_output = v.fit_transform(texts)
print(transformed_output.shape)

(48661, 10000)


In [ ]:
# print(v.vocabulary_)

{'cant': np.int64(1501), 'reset': np.int64(7421), 'user': np.int64(9297), 'agent': np.int64(639), 'installed': np.int64(4598), 'download': np.int64(2785), 'addons': np.int64(604), 'since': np.int64(8019), 'think': np.int64(8736), 'use': np.int64(9269), 'firefox': np.int64(3588), '52': np.int64(369), 'tried': np.int64(8943), 'didnt': np.int64(2596), 'work': np.int64(9790), 'setting': np.int64(7912), 'minimum': np.int64(5631), 'version': np.int64(9460), '53': np.int64(371), 'removing': np.int64(7283), 'resetting': np.int64(7422), 'via': np.int64(9524), 'none': np.int64(6000), 'value': np.int64(9418), 'modified': np.int64(5679), 'idea': np.int64(4337), 'user agent': np.int64(9303), 'didnt work': np.int64(2600), 'long': np.int64(5306), 'string': np.int64(8342), 'input': np.int64(4566), 'commandline': np.int64(1878), 'fail': np.int64(3380), 'cause': np.int64(1566), 'failure': np.int64(3400), 'filename': np.int64(3549), 'text': np.int64(8686), 'creating': np.int64(2232), 'add': np.int64(552)

In [17]:
all_feature_names = v.get_feature_names_out()

for word in all_feature_names:
  indx = v.vocabulary_.get(word)
  if v.idf_[indx] < 4:
    print(f"{word} {v.idf_[indx]}")

add 3.1248016493122748
also 3.60368685336475
bug 3.88868808298146
change 3.6813257137268924
code 3.3227617987154012
could 3.5830732335380477
create 3.7909637389144897
data 3.8312835152941145
doesnt 3.8574241771967186
dont 3.846390073433084
error 2.9634273622817764
example 3.5879818880628145
expected 3.7872866496969624
feature 3.995362443466153
file 3.027663386844522
following 3.7261322265436316
function 3.8983356531720017
get 3.1770642037411947
help 3.9965957426975023
hi 3.8714809954266105
id 3.9570744703436604
im 3.6020220361100863
image 3.907700771253812
instead 3.9682077861360066
issue 2.970184144744656
like 3.125489999021093
line 3.7108695100786244
link 3.998242511287024
make 3.4445894886053594
name 3.649717706581359
need 3.329283332294895
new 3.3231812618088
one 3.4165633665753883
page 3.687346201508475
please 3.608977134392968
possible 3.9839244102692266
problem 3.6335650623457147
project 3.9750282739602563
reproduce 3.930541505350877
request 3.794990589993525
run 3.7467449747428

In [ ]:
idf_df = pd.DataFrame({'word': all_feature_names, 'idf': v.idf_})

,word,idf
3050,error,2.963427
9460,version,2.969153
4746,issue,2.970184
9269,use,2.977878
3479,file,3.027663
9353,using,3.064876
552,add,3.124802
5146,like,3.125490
9843,would,3.134829
3866,get,3.177064


In [20]:
# plt.hist(idf_df['idf'], bins=50)
# plt.title("Distribution of IDF Values")
# plt.xlabel("IDF")
# plt.ylabel("Word Count")
# plt.show()

In [32]:
threshold = np.percentile(idf_df['idf'], 10) # bottom 10%
common_words = idf_df[idf_df['idf'] <= threshold].sort_values('idf', ascending=True)

print(f"Common words: {len(common_words)}")

Common words: 1003


In [19]:
common_words.info

<bound method DataFrame.info of                word       idf
3050          error  2.963427
9460        version  2.969153
4746          issue  2.970184
9269            use  2.977878
3479           file  3.027663
...             ...       ...
1059     automation  7.083124
1276         border  7.083124
2552      detection  7.083124
4396  im submitting  7.083124
4012        graphic  7.083124

[2005 rows x 2 columns]>

In [ ]:
# top_common = common_words.head(30)

# plt.figure(figsize=(12, 6))
# plt.barh(top_common['word'], top_common['idf'], color='skyblue')
# plt.xlabel('IDF Value (lower = more common)')
# plt.ylabel('Word')
# plt.title('Top 30 Most Common Words in GitHub Issues')
# plt.gca().invert_yaxis()
# plt.tight_layout()
# plt.show()

In [21]:
# Rule-based heuristic classifier
def assign_priority(text):
  text = str(text).lower()
  
  keywords = {
    "Critical": [
      "crash", "urgent", "block", "security", "data loss", "immediate", "leak", 
      "vulnerability", "malware", "corrupt", "unusable", "fatal", "system down",
      "cannot start", "inaccessible", "critical failure", "no access", "breach", 
      "exploit", "does not start", "infinite loop", "freeze system", "freezes"
      ],
    "High": [
      "fail", "error", "exception", "bug", "does not load", "timeout", "service down",
      "not working", "unable to", "stuck", "timeout", "incorrect", "wrong output",
      "not responding", "malfunction", "breaks", "regression", "unhandled",
      "unexpected", "missing file", "authentication", "deployment issue", "api error"
      ],
    "Medium": [
      "slow", "warning", "minor", "performance", "degraded", "inconsistent",
      "optimization", "delay", "unreliable", "resource usage", "partial failure",
      "sometimes", "temporary issue", "reduces performance", "fine-tuning", "occasionally",
      "lag", "refactor", "intermittent", "unreliable", "deprecated", "update", "load time"
      ],
    "Low": [
      "typo", "cosmetic", "suggestion", "enhancement", "refactor", "documentation",
      "readme", "cleanup", "alignment", "spacing", "grammar", "style", "formatting",
      "ui polish", "code comment", "naming", "color", "minor ui", "ui", "comment",
      "spelling", "non-critical", "translation", "ux", "improve"
      ]
  }
  
  for priority, words in keywords.items():
    if any(word in text for word in words):
      return priority
    
  return None

In [22]:
git_issues_df["priority"] = git_issues_df["clean_text"].apply(assign_priority)
git_issues_df.columns

Index(['issue_url', 'issue_title', 'body', 'clean_text', 'priority'], dtype='object')

In [23]:
git_issues_df["priority"].value_counts(dropna=False)

priority
None        20911
High        11772
Low          8137
Medium       4063
Critical     3778
Name: count, dtype: int64

In [24]:
# del transformed_output
# import gc
# gc.collect()

In [25]:
sample_df = git_issues_df.dropna(subset=['priority']).sample(n=5000, random_state=42)

In [26]:
# for priority in ['Critical', 'High', 'Medium', 'Low']:
#   text = " ".join(git_issues_df[git_issues_df['priority']==priority]['clean_text'].tolist())
#   wc = WordCloud(width=800, height=400, background_color='white').generate(text)
  
#   plt.figure(figsize=(10,5))
#   plt.imshow(wc, interpolation='bilinear')
#   plt.axis('off')
#   plt.title(f"Word Cloud for {priority} Issues")
#   plt.show()

In [27]:
for priority in ['Critical', 'High', 'Medium', 'Low']:
  words = " ".join(sample_df[sample_df['priority']==priority]['clean_text']).split()
  top_words = Counter(words).most_common(15)
  print(f"{priority} top words: {top_words}\n")

Critical top words: [('error', 376), ('crash', 354), ('1', 325), ('version', 292), ('block', 274), ('issue', 270), ('file', 240), ('line', 186), ('code', 171), ('2', 158), ('using', 152), ('get', 143), ('user', 138), ('like', 137), ('use', 122)]

High top words: [('error', 2133), ('file', 1048), ('version', 959), ('issue', 829), ('1', 681), ('bug', 659), ('line', 633), ('code', 534), ('test', 529), ('get', 526), ('use', 522), ('using', 492), ('type', 463), ('2', 440), ('function', 409)]

Medium top words: [('update', 409), ('issue', 280), ('version', 235), ('would', 175), ('file', 167), ('add', 153), ('new', 143), ('use', 134), ('change', 132), ('need', 128), ('one', 128), ('using', 118), ('page', 116), ('like', 113), ('get', 112)]

Low top words: [('use', 308), ('file', 307), ('need', 304), ('would', 299), ('like', 280), ('add', 274), ('version', 274), ('using', 272), ('work', 254), ('issue', 248), ('new', 237), ('build', 229), ('make', 208), ('user', 207), ('1', 199)]



In [28]:
processed_df = git_issues_df.dropna(subset=['priority']).copy()

joblib.dump(processed_df, '../../data/processed/precessed_github_issues_df.joblib')
print("Processed DataFrame saved successfully!")

Processed DataFrame saved successfully!


In [29]:
v = TfidfVectorizer(
  max_features=10000,
  ngram_range=(1, 2),
  min_df=5,
  max_df=0.9
)
X = v.fit_transform(processed_df['clean_text'])
y = processed_df['priority']

In [30]:
joblib.dump(X, '../../data/processed/tfidf/X_tfidf.joblib')
joblib.dump(y, '../../data/processed/tfidf/y_labels.joblib')
joblib.dump(v, '../../data/processed/tfidf/tfidf_vectorizer.joblib')
print("TF-IDF matrix, labels, and vectorizer saved successfully!")

TF-IDF matrix, labels, and vectorizer saved successfully!
